# QA simulation of vacancies in graphene explained

## The model

In this tutorial, we are going to use a simple model to calculate the energy of vacancies in graphene.

**The material**

The chemical system we are going to study is a graphene sheet. In order to be able to calculate the energy of up to three vacancies, we are going to use a 3x3 supercell of the primitive cell.

Let's first build the system. For this task we are going to be using a python library called pymatgen. pymatgen is a very powerful tool developed as part of the Materials Project. It can be used to create, modify, analyse 3D systems. Don't worry if you are not familiar with pymatgen, we are only going to use it to generate the structure and every step is explained.

First we need to import the two objects that we are going to use to generate the graphene structure, which is called Structure:

In [1]:
from pymatgen.core.structure import Structure

We are going to use numpy to define the lattice as a 3x3 matrix:

In [4]:
import numpy as np

lattice = np.array([[ 1.233862, -2.137112,  0.      ],
                   [ 1.233862,  2.137112,  0.      ],
                   [ 0.      ,  0.      ,  8.685038]])

The Structure object requires three pieces on informations: the lattice, the species and the fractional coordinates of these species in the cell defined by lattice. So, we are going to create and inspect what information is in the graphene object by displaying it.

In [12]:
graphene = Structure(lattice, species=['C','C'], coords=[[2/3, 1/3, 0. ],[1/3, 2/3, 0.]])
display(graphene)

Structure Summary
Lattice
    abc : 2.4677242827325747 2.4677242827325747 8.685038
 angles : 90.0 90.0 120.00000758003493
 volume : 45.80317575176706
      A : 1.233862 -2.137112 0.0
      B : 1.233862 2.137112 0.0
      C : 0.0 0.0 8.685038
PeriodicSite: C (1.2339, -0.7124, 0.0000) [0.6667, 0.3333, 0.0000]
PeriodicSite: C (1.2339, 0.7124, 0.0000) [0.3333, 0.6667, 0.0000]

The supercell can be created by using

In [ ]:
n_supercell = 3
scaling_matrix = np.identity(3)*n_supercell
scaling_matrix[2][2] = 1
graphene_supercell = copy.deepcopy(graphene)
graphene_supercell.make_supercell(scaling_matrix)

## Classical approach

## Quantum annealing